In [1]:
import math as m
import numpy as np
import matplotlib as mpl
from PIL import Image as im
import matplotlib.pyplot as plt
import tensorflow.keras as keras
import sklearn.datasets as datasets

TEMPFILE_NAME = "temp.png"

def save_to_gif(filename, images, duration=100):
    images[0].save(
        filename,
        optimize=False,
        save_all=True,
        append_images=images[1:],
        loop=0,
        duration=duration,
    )

np.random.seed(1)

COLORS = np.array(['purple', 'green', 'blue'])

# custom CMAP
cvals  = [0, .5, 1]
colors = ['blue', 'white', 'purple']
norm=plt.Normalize(min(cvals),max(cvals))
tuples = list(zip(map(norm,cvals), colors))
CMAP = mpl.colors.LinearSegmentedColormap.from_list("", tuples, 100)

cvals  = [0, .5, 1]
colors = ['white', 'white', 'purple']
norm=plt.Normalize(min(cvals),max(cvals))
tuples = list(zip(map(norm,cvals), colors))
CMAP_PURPLE = mpl.colors.LinearSegmentedColormap.from_list("", tuples, 100)

cvals  = [0, .5, 1]
colors = ['white', 'white', 'green']
norm=plt.Normalize(min(cvals),max(cvals))
tuples = list(zip(map(norm,cvals), colors))
CMAP_GREEN = mpl.colors.LinearSegmentedColormap.from_list("", tuples, 100)

cvals  = [0, .5, 1]
colors = ['white', 'white', 'blue']
norm=plt.Normalize(min(cvals),max(cvals))
tuples = list(zip(map(norm,cvals), colors))
CMAP_BLUE = mpl.colors.LinearSegmentedColormap.from_list("", tuples, 100)

CENTERS = [[0, 0]]
STDEV = 1
DATA, _ = datasets.make_blobs(
        n_samples=200,
        centers=CENTERS,
        cluster_std=STDEV,
        random_state=1
    )
xlim_min = CENTERS[0][0] - 3 * STDEV
xlim_max = CENTERS[0][0] + 3 * STDEV
ylim_min = CENTERS[0][1] - 3 * STDEV
ylim_max = CENTERS[0][1] + 3 * STDEV

def generate_line_data(t, w1, w2, b):
    X = np.array(list(filter(lambda x : w1 * x[0] + w2 * x[1] + b < -.5 or w1 * x[0] + w2 * x[1] + b > .5, t)))
    Y = np.array([0 if w1 * x[0] + w2 * x[1] + b >= 0 else 2 for x in X])
    return X, Y

def sigmoid(x):
    e = np.exp(-x)
    return 1 / (1 + e)

def generate_3_classes_1D(SIZE):
    X = np.linspace(0, 5 * SIZE, SIZE) + np.random.randn(SIZE) * 4
    Y = np.array([0 if x < 18 else 1 if x < 60 else 2 for x in X])
    return X, Y

def generate_3_classes_2D(t, w1, w2):
    f1 = lambda x1, x2: w1[0] * x1 + w1[1] * x2
    f2 = lambda x1, x2: w2[0] * x1 + w2[1] * x2
    X = np.array(list(filter(lambda x : (f1(x[0], x[1]) < -.5 or f1(x[0], x[1]) > .5) and (f2(x[0], x[1]) < -.5 or f2(x[0], x[1]) > .5), t)))
    Y = np.array([0 if f1(x[0], x[1]) >= 0 and f2(x[0], x[1]) >= 0 else 1 if (f1(x[0], x[1]) < 0 and f2(x[0], x[1]) >= 0) else 2 for x in X])
    return X, Y

# OTHER FUNCTIONS
def generate_circle_data(t):
    X = np.array(list(filter(lambda x : (x[0] - CENTERS[0][0])**2 + (x[1] - CENTERS[0][1])**2 < 1 or (x[0] - CENTERS[0][0])**2 + (x[1] - CENTERS[0][1])**2 > 1.5, t)))
    Y = np.array([0 if (x[0] - CENTERS[0][0])**2 + (x[1] - CENTERS[0][1])**2 >= 1 else 2 for x in X])
    return X, Y

def generate_square_data(t):
    X = np.array(list(filter(lambda x : x[0]**2 - x[1] < .4 or x[0]**2 - x[1] > 1.1, t)))
    Y = np.array([1 if x[0]**2 - x[1] >= .75 else 0 for x in X])
    return X, Y

def generate_curve_data(t):
    X = np.array(list(filter(lambda x : m.cos(4*x[0]) - x[1] < -.5 or m.cos(4*x[0]) - x[1] > .5, t)))
    Y = np.array([1 if m.cos(4*x[0]) - x[1] >= 0 else 0 for x in X])
    return X, Y

## Classification

In [2]:
X, Y = generate_line_data(DATA, 1, -1, 0)

xplot = np.linspace(-3, 3, 100)
plt.scatter(X[:,0],X[:,1],color=COLORS[Y].tolist(), s=400, alpha=.5)
plt.xlim(xlim_min, xlim_max)
plt.ylim(ylim_min, ylim_max)
plt.xlabel("$x_1$", fontsize=15)
plt.ylabel("$x_2$", fontsize=15)
plt.savefig("000.png")
plt.close()

## Class 0

In [3]:
X, Y = generate_line_data(DATA, 1, -1, 0)

class0 = np.where(Y == 0)
xplot = np.linspace(-3, 3, 100)
plt.scatter(X[:,0],X[:,1],color=COLORS[Y].tolist(), s=400, alpha=.5)
plt.scatter(X[class0][:,0],X[class0][:,1],color=COLORS[Y[class0]].tolist(), s=400, alpha=1)
plt.xlim(xlim_min, xlim_max)
plt.ylim(ylim_min, ylim_max)
plt.xlabel("$x_1$", fontsize=15)
plt.ylabel("$x_2$", fontsize=15)
plt.savefig("001.png")
plt.close()

## Class 1

In [4]:
X, Y = generate_line_data(DATA, 1, -1, 0)

class1 = np.where(Y == 1)
xplot = np.linspace(-3, 3, 100)
plt.scatter(X[:,0],X[:,1],color=COLORS[Y].tolist(), s=400, alpha=.5)
plt.scatter(X[class1][:,0],X[class1][:,1],color=COLORS[Y[class1]].tolist(), s=400, alpha=1)
plt.xlim(xlim_min, xlim_max)
plt.ylim(ylim_min, ylim_max)
plt.xlabel("$x_1$", fontsize=15)
plt.ylabel("$x_2$", fontsize=15)
plt.savefig("002.png")
plt.close()

## X_1

In [5]:
X, Y = generate_line_data(DATA, 1, -1, 0)

xplot = np.linspace(-3, 3, 100)
fig, ax = plt.subplots()
ax.scatter(X[:,0],X[:,1],color=COLORS[Y].tolist(), s=400, alpha=.5)
ax.set_xlim(xlim_min, xlim_max)
ax.set_ylim(ylim_min, ylim_max)
ax.set_xlabel("$x_1$", fontsize=15, fontweight='bold', c='r')
ax.set_ylabel("$x_2$", fontsize=15)
fig.savefig("003.png")
plt.close()

## X_2

In [6]:
X, Y = generate_line_data(DATA, 1, -1, 0)

xplot = np.linspace(-3, 3, 100)
fig, ax = plt.subplots()
ax.scatter(X[:,0],X[:,1],color=COLORS[Y].tolist(), s=400, alpha=.5)
ax.set_xlim(xlim_min, xlim_max)
ax.set_ylim(ylim_min, ylim_max)
ax.set_xlabel("$x_1$", fontsize=15)
ax.set_ylabel("$x_2$", fontsize=15, fontweight='bold', c='r')
fig.savefig("004.png")
plt.close()

## Random Line

In [7]:
params = [
    [1, 1],
    [2, 3],
    [-1, 1],
    [3, -1],
    [-5, -2]
]

def snap_line(w1, w2):
    xplot = np.linspace(-3, 3, 100)
    fig, ax = plt.subplots()
    ax.scatter(X[:,0], X[:,1], color=COLORS[Y].tolist(), s=400, alpha=.5)
    ax.plot(xplot, w1 * xplot + w2 * xplot, c='b', linestyle='dashed')
    ax.set_xlim(xlim_min, xlim_max)
    ax.set_ylim(ylim_min, ylim_max)
    ax.set_xlabel("$x_1$", fontsize=15)
    ax.set_ylabel("$x_2$", fontsize=15)
    fig.savefig(TEMPFILE_NAME, bbox_inches='tight')
    plt.close()

    return im.fromarray(np.asarray(im.open(TEMPFILE_NAME)))

images = []
for p in params:
    images.append(snap_line(p[0], p[1]))

save_to_gif("005.gif", images, 500)

## Line

In [8]:
X, Y = generate_line_data(DATA, 1, -1, 0)

xplot = np.linspace(-3, 3, 100)
plt.scatter(X[:,0],X[:,1],color=COLORS[Y].tolist(), s=400, alpha=.5)
plt.plot(xplot, xplot, c='b', linestyle='dashed', label="$x_1 - x_2 = 0$")
plt.xlim(xlim_min, xlim_max)
plt.ylim(ylim_min, ylim_max)
plt.xlabel("$x_1$", fontsize=15)
plt.ylabel("$x_2$", fontsize=15)
plt.legend(fontsize=18)
plt.savefig("006.png")
plt.close()

## On the line

In [9]:
X, Y = generate_line_data(DATA, 1, -1, 0)

xplot = np.linspace(-3, 3, 100)
plt.scatter(X[:,0],X[:,1],color=COLORS[Y].tolist(), s=400, alpha=.5)
plt.plot(xplot, xplot, c='b', linestyle='dashed', label="$x_1 - x_2 = 0$")
plt.scatter(0, 0, color='g', s=400, alpha=1)
plt.xlim(xlim_min, xlim_max)
plt.ylim(ylim_min, ylim_max)
plt.xlabel("$x_1$", fontsize=15)
plt.ylabel("$x_2$", fontsize=15)
plt.legend(fontsize=15)
plt.savefig("007.png")
plt.close()

## Not on the line

In [10]:
X, Y = generate_line_data(DATA, 1, -1, 0)

f = lambda x, y: x - y
x_point = [2, 1]

xplot = np.linspace(-3, 3, 100)
plt.scatter(X[:,0],X[:,1],color=COLORS[Y].tolist(), s=400, alpha=.5)
plt.plot(xplot, xplot, c='b', linestyle='dashed', label="$x_1 - x_2 = 0$")
plt.scatter(x_point[0], x_point[1], color='g', s=400, alpha=1, label=f'{x_point[0]:.1f}' + " - " + f'{x_point[1]:.1f}' + ' = ' + f'{f(x_point[0], x_point[1]):.1f}')
plt.xlim(xlim_min, xlim_max)
plt.ylim(ylim_min, ylim_max)
plt.xlabel("$x_1$", fontsize=15)
plt.ylabel("$x_2$", fontsize=15)
plt.legend(fontsize=15, loc='upper left')
plt.savefig("008.png")
plt.close()

## All points with 1

In [11]:
X, Y = generate_line_data(DATA, 1, -1, 0)

f = lambda x, y: x - y
x_point = [2, 1]

xplot = np.linspace(-3, 3, 100)
plt.scatter(X[:,0],X[:,1],color=COLORS[Y].tolist(), s=400, alpha=.5)
plt.plot(xplot, xplot, c='b', linestyle='dashed', label="$x_1 - x_2 = 0$")
plt.scatter(x_point[0], x_point[1], color='g', s=400, alpha=1)
plt.plot(xplot, xplot - x_point[0] + x_point[1], c='g', linestyle='dashed', label="$x_1 - x_2 = $" + f'{f(x_point[0],x_point[1]):.1f}')
plt.xlim(xlim_min, xlim_max)
plt.ylim(ylim_min, ylim_max)
plt.xlabel("$x_1$", fontsize=15)
plt.ylabel("$x_2$", fontsize=15)
plt.legend(fontsize=15, loc='upper left')
plt.savefig("009.png")
plt.close()

In [12]:
X, Y = generate_line_data(DATA, 1, -1, 0)

f = lambda x, y: x - y
x_point = [2, 1]

xplot = np.linspace(-3, 3, 100)
plt.scatter(X[:,0],X[:,1],color=COLORS[Y].tolist(), s=400, alpha=.5)
plt.plot(xplot, xplot, c='b', linestyle='dashed', label="$x_1 - x_2 = 0$")
plt.plot(xplot, xplot - 1, c='g', linestyle='dashed')
plt.xlim(xlim_min, xlim_max)
plt.ylim(ylim_min, ylim_max)
plt.xlabel("$x_1$", fontsize=15)
plt.ylabel("$x_2$", fontsize=15)
plt.legend(fontsize=15, loc='upper left')
plt.savefig("010.png")
plt.close()

## All points 2

In [13]:
X, Y = generate_line_data(DATA, 1, -1, 0)

f = lambda x, y: x - y - 2

xplot = np.linspace(-3, 3, 100)
plt.scatter(X[:,0],X[:,1],color=COLORS[Y].tolist(), s=400, alpha=.5)
plt.plot(xplot, xplot, c='b', linestyle='dashed', label="$x_1 - x_2 = 0$")
plt.plot(xplot, xplot - 1, c='g', linestyle='dashed')
plt.plot(xplot, xplot - 2, c='g', linestyle='dashed')
plt.xlim(xlim_min, xlim_max)
plt.ylim(ylim_min, ylim_max)
plt.xlabel("$x_1$", fontsize=15)
plt.ylabel("$x_2$", fontsize=15)
plt.legend(fontsize=15, loc='upper left')
plt.savefig("011.png")
plt.close()

## All points 3

In [14]:
X, Y = generate_line_data(DATA, 1, -1, 0)

f = lambda x, y: x - y - 3

xplot = np.linspace(-3, 3, 100)
plt.scatter(X[:,0],X[:,1],color=COLORS[Y].tolist(), s=400, alpha=.5)
plt.plot(xplot, xplot, c='b', linestyle='dashed', label="$x_1 - x_2 = 0$")
plt.plot(xplot, xplot - 1, c='g', linestyle='dashed')
plt.plot(xplot, xplot - 2, c='g', linestyle='dashed')
plt.plot(xplot, xplot - 3, c='g', linestyle='dashed')
plt.xlim(xlim_min, xlim_max)
plt.ylim(ylim_min, ylim_max)
plt.xlabel("$x_1$", fontsize=15)
plt.ylabel("$x_2$", fontsize=15)
plt.legend(fontsize=15, loc='upper left')
plt.savefig("012.png")
plt.close()

## All points positive

In [15]:
X, Y = generate_line_data(DATA, 1, -1, 0)

xplot = np.linspace(-3, 3, 100)
x, y = np.meshgrid(xplot, xplot)
mask = x - y > 0
x_masked = np.ma.masked_where(~mask, x)
y_masked = np.ma.masked_where(~mask, y)

fig, ax = plt.subplots()
ax.scatter(X[:,0],X[:,1],color=COLORS[Y].tolist(), s=400, alpha=.5)
boundary, = ax.plot(xplot, xplot, c='b', linestyle='dashed', label="$x_1 - x_2 = 0$")
ax.contourf(x_masked, y_masked, x_masked - y_masked > 0, alpha=.3, cmap=CMAP)
ax.set_xlim(xlim_min, xlim_max)
ax.set_ylim(ylim_min, ylim_max)
ax.set_xlabel("$x_1$", fontsize=15)
ax.set_ylabel("$x_2$", fontsize=15)
purple_patch = mpl.patches.Patch(color=mpl.cm.get_cmap(CMAP)(.7), alpha=.5, label='$x_1 - x_2 > 0$')
ax.legend(handles=[boundary, purple_patch], fontsize=15, loc='upper left')
fig.savefig("013_positive.png")
plt.close()

/var/folders/89/x_333x1920nb7086021gwhlw0000gn/T/ipykernel_17100/300066274.py:17: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  purple_patch = mpl.patches.Patch(color=mpl.cm.get_cmap(CMAP)(.7), alpha=.5, label='$x_1 - x_2 > 0$')


In [16]:
X, Y = generate_line_data(DATA, 1, -1, 0)

xplot = np.linspace(-3, 3, 100)
x, y = np.meshgrid(xplot, xplot)
mask = x - y < 0
x_masked = np.ma.masked_where(~mask, x)
y_masked = np.ma.masked_where(~mask, y)

fig, ax = plt.subplots()
ax.scatter(X[:,0],X[:,1],color=COLORS[Y].tolist(), s=400, alpha=.5)
boundary, = ax.plot(xplot, xplot, c='b', linestyle='dashed', label="$x_1 - x_2 = 0$")
ax.contourf(x_masked, y_masked, x_masked - y_masked > 0, alpha=.3, cmap=CMAP)
ax.set_xlim(xlim_min, xlim_max)
ax.set_ylim(ylim_min, ylim_max)
ax.set_xlabel("$x_1$", fontsize=15)
ax.set_ylabel("$x_2$", fontsize=15)
purple_patch = mpl.patches.Patch(color=mpl.cm.get_cmap(CMAP)(.3), alpha=.5, label='P(blue) > 1/2')
ax.legend(handles=[boundary, purple_patch], fontsize=15, loc='upper left')
fig.savefig("013_negative.png")
plt.close()

/var/folders/89/x_333x1920nb7086021gwhlw0000gn/T/ipykernel_17100/2178898089.py:17: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  purple_patch = mpl.patches.Patch(color=mpl.cm.get_cmap(CMAP)(.3), alpha=.5, label='P(blue) > 1/2')


## Closer to the line

In [17]:
X, Y = generate_line_data(DATA, 1, -1, 0)

f = lambda x, y: x - y
point2 = [1.5, -2.5]

xplot = np.linspace(-3, 3, 100)
plt.scatter(X[:,0],X[:,1],color=COLORS[Y].tolist(), s=400, alpha=.5)

plt.scatter(point2[0], point2[1], color='#BF5700', s=400, alpha=1, edgecolor='black')
plt.text(point2[0] + .25, point2[1] + .25, s=f'{f(point2[0], point2[1]):.1f}', fontsize=15)

plt.plot(xplot, xplot, c='b', linestyle='dashed', label="$x_1 - x_2 = 0$")

plt.xlim(xlim_min, xlim_max)
plt.ylim(ylim_min, ylim_max)
plt.xlabel("$x_1$", fontsize=15)
plt.ylabel("$x_2$", fontsize=15)
plt.legend(fontsize=15, facecolor='#BF5700', framealpha=1, edgecolor='black', labelcolor='white')
plt.savefig("014.png")
plt.close()

In [18]:
X, Y = generate_line_data(DATA, 1, -1, 0)

f = lambda x, y: x - y
point1 = [2, 1]
point2 = [1.5, -2.5]

xplot = np.linspace(-3, 3, 100)
plt.scatter(X[:,0],X[:,1],color=COLORS[Y].tolist(), s=400, alpha=.5)
plt.scatter(point1[0], point1[1], color='#BF5700', s=400, alpha=1, edgecolor='black')
plt.text(point1[0] + .25, point1[1] + .25, s=f'{f(point1[0], point1[1]):.1f}', fontsize=15)

plt.scatter(point2[0], point2[1], color='#BF5700', s=400, alpha=1, edgecolor='black')
plt.text(point2[0] + .25, point2[1] + .25, s=f'{f(point2[0], point2[1]):.1f}', fontsize=15)

plt.plot(xplot, xplot, c='b', linestyle='dashed', label="$x_1 - x_2 = 0$")

plt.xlim(xlim_min, xlim_max)
plt.ylim(ylim_min, ylim_max)
plt.xlabel("$x_1$", fontsize=15)
plt.ylabel("$x_2$", fontsize=15)
plt.legend(fontsize=15, facecolor='#BF5700', framealpha=1, edgecolor='black', labelcolor='white')
plt.savefig("015.png")
plt.close()

In [19]:
X, Y = generate_line_data(DATA, 1, -1, 0)

f = lambda x, y : 4*x - 4*y

point2 = [2, 1]

xplot = np.linspace(-3, 3, 100)
plt.scatter(X[:,0],X[:,1],color=COLORS[Y].tolist(), s=400, alpha=.5)

plt.scatter(point2[0], point2[1], color='#A1CCA5', s=400, alpha=1, edgecolor='black')
plt.text(point2[0] + .2, point2[1] + .2, s=f'{f(point2[0], point2[1]):.1f}', fontsize=15)

plt.plot(xplot, xplot, c='b', linestyle='dashed', label="$4x_1 - 4x_2 = 0$")

plt.xlim(xlim_min, xlim_max)
plt.ylim(ylim_min, ylim_max)
plt.xlabel("$x_1$", fontsize=15)
plt.ylabel("$x_2$", fontsize=15)
plt.legend(fontsize=15, facecolor='#A1CCA5', framealpha=1, edgecolor='black')
plt.savefig("016.png")
plt.close()

In [20]:
def f(x, y):
    return x - y

X, Y = generate_line_data(DATA, 1, -1, 0)

xplot = np.linspace(-3, 3, 100)

for i in range(len(X)):
    point = X[i]
    plt.text(point[0], point[1], s=f'{f(point[0], point[1]):.1f}', fontsize=int(20 * sigmoid(abs(f(point[0], point[1])))), color=COLORS[Y[i]])

plt.plot(xplot, xplot, c='b', linestyle='dashed', label="$x_1 - x_2 = 0$")
plt.xlim(xlim_min, xlim_max)
plt.ylim(ylim_min, ylim_max)
plt.xlabel("$x_1$", fontsize=15)
plt.ylabel("$x_2$", fontsize=15)
plt.legend(fontsize=15)
plt.savefig("017.png")
plt.close()

## Many Lines

In [21]:
X, Y = generate_line_data(DATA, 1, -1, 0)

xplot = np.linspace(-1.5, 1.5, 100)
plt.scatter(X[:,0],X[:,1],color=COLORS[Y].tolist(), s=400, alpha=.5)
plt.plot(2 * xplot, 2 * xplot, c='b', linestyle='dashed', label="$2x_1 - 2x_2 = 0$")
plt.xlim(xlim_min, xlim_max)
plt.ylim(ylim_min, ylim_max)
plt.xlabel("$x_1$", fontsize=15)
plt.ylabel("$x_2$", fontsize=15)
plt.legend(fontsize=15)
plt.savefig("018.png")
plt.close()

In [22]:
X, Y = generate_line_data(DATA, 1, -1, 0)

xplot = np.linspace(-6, 6, 100)
plt.scatter(X[:,0],X[:,1],color=COLORS[Y].tolist(), s=400, alpha=.5)
plt.plot(.5 * xplot, .5 * xplot, c='b', linestyle='dashed', label="$\\frac{1}{2}x_1 - \\frac{1}{2}x_2 = 0$")
plt.xlim(xlim_min, xlim_max)
plt.ylim(ylim_min, ylim_max)
plt.xlabel("$x_1$", fontsize=15)
plt.ylabel("$x_2$", fontsize=15)
plt.legend(fontsize=15)
plt.savefig("019.png")
plt.close()

## Not on line diff params

In [23]:
def f(x1, x2):
    return x1 - x2

X, Y = generate_line_data(DATA, 1, -1, 0)

np.random.seed(1)
index = np.random.choice(range(len(X)))
x_point = X[index]
x_point_color = COLORS[Y[index]]

xplot = np.linspace(-6, 6, 100)
plt.scatter(X[:,0], X[:,1], color=COLORS[Y].tolist(), s=400, alpha=.5)
plt.plot(.5 * xplot, .5 * xplot, c='b', linestyle='dashed', label="$x_1 - x_2 = 0$")
plt.scatter(x_point[0], x_point[1], color='g', s=400, alpha=1)
plt.text(x_point[0] + .25, x_point[1] + .25, s=f'{f(x_point[0],x_point[1]):.2f}', fontsize=20, fontweight='bold')
plt.xlim(xlim_min, xlim_max)
plt.ylim(ylim_min, ylim_max)
plt.xlabel("$x_1$", fontsize=15)
plt.ylabel("$x_2$", fontsize=15)
plt.legend(fontsize=13)
plt.savefig("020.png")
plt.close()

In [24]:
C = 1/2
def f(x1, x2):
    return C * x1 - C * x2

X, Y = generate_line_data(DATA, 1, -1, 0)

xplot = np.linspace(-3, 3, 100)

for i in range(len(X)):
    point = X[i]
    plt.text(point[0], point[1], s=f'{f(point[0], point[1]):.1f}', fontsize=int(20 * sigmoid(abs(f(point[0], point[1])))), color=COLORS[Y[i]])

plt.plot(xplot, xplot, c='b', linestyle='dashed', label="$\\frac{1}{2}x_1 - \\frac{1}{2}x_2 = 0$")

plt.xlim(xlim_min, xlim_max)
plt.ylim(ylim_min, ylim_max)
plt.xlabel("$x_1$", fontsize=15)
plt.ylabel("$x_2$", fontsize=15)
plt.legend(fontsize=15)
plt.savefig("021.png")
plt.close()

In [25]:
C = 2
def f(x1, x2):
    return C * x1 - C * x2

X, Y = generate_line_data(DATA, 1, -1, 0)

xplot = np.linspace(-3, 3, 100)

for i in range(len(X)):
    point = X[i]
    plt.text(point[0], point[1], s=f'{f(point[0], point[1]):.1f}', fontsize=int(20 * sigmoid(abs(f(point[0], point[1])))), color=COLORS[Y[i]])

plt.plot(xplot, xplot, c='b', linestyle='dashed', label="$2x_1 - 2x_2 = 0$")

plt.xlim(xlim_min, xlim_max)
plt.ylim(ylim_min, ylim_max)
plt.xlabel("$x_1$", fontsize=15)
plt.ylabel("$x_2$", fontsize=15)
plt.legend(fontsize=15)
plt.savefig("022.png")
plt.close()

In [26]:
def f(x1, x2):
    return x1 - x2

X, Y = generate_line_data(DATA, 1, -1, 0)

xplot = np.linspace(-3, 3, 100)

for i in range(len(X)):
    point = X[i]
    if Y[i] == 0:
        plt.text(point[0], point[1], s="$< \\frac{1}{2}$", fontsize=10, color=COLORS[Y[i]])
    else:
        plt.text(point[0], point[1], s="$> \\frac{1}{2}$", fontsize=10, color=COLORS[Y[i]])


plt.text(0, 0, s="$= \\frac{1}{2}$", fontsize=10, color='g')
plt.text(-1, -1, s="$= \\frac{1}{2}$", fontsize=10, color='g')
plt.text(1, 1, s="$= \\frac{1}{2}$", fontsize=10, color='g')
plt.text(-2, -2, s="$= \\frac{1}{2}$", fontsize=10, color='g')
plt.text(2, 2, s="$= \\frac{1}{2}$", fontsize=10, color='g')

plt.plot(xplot, xplot, c='b', linestyle='dashed', label="$x_1 - x_2 = 0$")

plt.xlim(xlim_min, xlim_max)
plt.ylim(ylim_min, ylim_max)
plt.xlabel("$x_1$", fontsize=15)
plt.ylabel("$x_2$", fontsize=15)
plt.legend(fontsize=15)
plt.savefig("023.png")
plt.close()

## diff params lines 123

In [27]:
X, Y = generate_line_data(DATA, 1, -1, 0)

xplot = np.linspace(-3, 3, 100)
plt.scatter(X[:,0],X[:,1],color=COLORS[Y].tolist(), s=400, alpha=.5)
plt.plot(xplot, xplot, c='b', linestyle='dashed', label="$2x_1 - 2x_2 = 0$")
plt.xlim(xlim_min, xlim_max)
plt.ylim(ylim_min, ylim_max)
plt.xlabel("$x_1$", fontsize=15)
plt.ylabel("$x_2$", fontsize=15)
plt.legend(fontsize=15, loc='upper left')
plt.savefig("024.png")
plt.close()

In [28]:
X, Y = generate_line_data(DATA, 1, -1, 0)

xplot = np.linspace(-3, 3, 100)
plt.scatter(X[:,0],X[:,1],color=COLORS[Y].tolist(), s=400, alpha=.5)
plt.plot(xplot, xplot, c='b', linestyle='dashed', label="$2x_1 - 2x_2 = 0$")
plt.plot(xplot, xplot - 1/2, c='g', linestyle='dashed')
plt.xlim(xlim_min, xlim_max)
plt.ylim(ylim_min, ylim_max)
plt.xlabel("$x_1$", fontsize=15)
plt.ylabel("$x_2$", fontsize=15)
plt.legend(fontsize=15, loc='upper left')
plt.savefig("025.png")
plt.close()

In [29]:
X, Y = generate_line_data(DATA, 1, -1, 0)

xplot = np.linspace(-3, 3, 100)
plt.scatter(X[:,0],X[:,1],color=COLORS[Y].tolist(), s=400, alpha=.5)
plt.plot(xplot, xplot, c='b', linestyle='dashed', label="$2x_1 - 2x_2 = 0$")
plt.plot(xplot, xplot - 1/2, c='g', linestyle='dashed')
plt.plot(xplot, xplot - 1, c='g', linestyle='dashed')
plt.xlim(xlim_min, xlim_max)
plt.ylim(ylim_min, ylim_max)
plt.xlabel("$x_1$", fontsize=15)
plt.ylabel("$x_2$", fontsize=15)
plt.legend(fontsize=15, loc='upper left')
plt.savefig("026.png")
plt.close()

In [30]:
X, Y = generate_line_data(DATA, 1, -1, 0)

xplot = np.linspace(-3, 3, 100)
plt.scatter(X[:,0],X[:,1],color=COLORS[Y].tolist(), s=400, alpha=.5)
plt.plot(xplot, xplot, c='b', linestyle='dashed', label="$x_1 - x_2 = 0$")
plt.plot(xplot, xplot - 1/2, c='g', linestyle='dashed')
plt.plot(xplot, xplot - 1, c='g', linestyle='dashed')
plt.plot(xplot, xplot - 1.5, c='g', linestyle='dashed')
plt.xlim(xlim_min, xlim_max)
plt.ylim(ylim_min, ylim_max)
plt.xlabel("$x_1$", fontsize=15)
plt.ylabel("$x_2$", fontsize=15)
plt.legend(fontsize=15, loc='upper left')
plt.savefig("027.png")
plt.close()

In [31]:
def snap(C):
    fig, ax = plt.subplots()
    ax.scatter(X[:,0], X[:,1], color=COLORS[Y].tolist(), s=400, alpha=.5)
    ax.plot(xplot, xplot, c='b', linestyle='dashed', label=r'$C x_1 - C x_2 = 0$'.replace("C", f'{C:.2f}'))
    ax.plot(xplot, xplot - 1/C, c='g', linestyle='dashed')
    ax.plot(xplot, xplot - 2/C, c='g', linestyle='dashed')
    ax.plot(xplot, xplot - 3/C, c='g', linestyle='dashed')
    ax.set_xlim(xlim_min, xlim_max)
    ax.set_ylim(ylim_min, ylim_max)
    ax.set_xlabel("$x_1$", fontsize=15)
    ax.set_ylabel("$x_2$", fontsize=15)
    ax.legend(fontsize=15, loc='upper left')
    fig.savefig(TEMPFILE_NAME)
    plt.close()

    return im.fromarray(np.asarray(im.open(TEMPFILE_NAME)))

images = []

for i in range(1,6):
    images.append(snap(i))

save_to_gif('028.gif', images, duration=500)

## Sigmoid

In [32]:
def f(x1, x2, C):
    return (1/C) * x1 - (1/C) * x2

X, Y = generate_line_data(DATA, 1, -1, 0)
xmax = 6

def snap(i):
    fig, ax = plt.subplots()
    if i == 0:
        xplot = 0
        plt.scatter(xplot, sigmoid(xplot), c='#CBC3E3', s=100)
    else:
        if i > 0:
            xplot = np.linspace(0, i, 100)
        if i < 0:
            xplot = np.linspace(i, xmax, 200)
        ax.plot(xplot, sigmoid(xplot), c='#CBC3E3', linewidth=4)
    ax.axvline(0, color='grey', linestyle='--', linewidth=0.5)
    ax.axhline(sigmoid(0), color='grey', linestyle='--', linewidth=0.5)
    ax.set_xticks(np.arange(-xmax, xmax+1, 2))

    y_ticks = np.append(np.arange(0, 1.1, 0.2), sigmoid(0))
    ax.set_xlabel('RESULT')
    ax.set_ylabel('PROBABILITY')
    ax.set_yticks(np.unique(np.sort(y_ticks)))
    ax.set_xlim(-xmax, xmax)
    ax.set_ylim(-.1, 1.1)
    fig.savefig(TEMPFILE_NAME)
    plt.close()

    return im.fromarray(np.asarray(im.open(TEMPFILE_NAME)))

images = []

for _ in range(10):
    images.append(snap(0))
for i in range(1, xmax * 10):
    images.append(snap(i/10))
for i in range(1, xmax * 10):
    images.append(snap(-i/10))

save_to_gif("029.gif", images, 100)

In [33]:
fig, ax = plt.subplots()
xplot = np.linspace(-xmax, xmax, 200)
ax.plot(xplot, sigmoid(xplot), c='#CBC3E3', linewidth=4, label='$\\frac{1}{1 + e^{-x}}$')
ax.axvline(0, ymin=0, ymax=(sigmoid(0)+.1) / 1.2, color='grey', linestyle='--', linewidth=1)
ax.axhline(sigmoid(0), xmin=0, xmax=1/2, color='grey', linestyle='--', linewidth=1)
ax.axhline(0, color='grey', linestyle='--', linewidth=0.5)
ax.axhline(1, color='grey', linestyle='--', linewidth=0.5)
ax.set_xticks(np.arange(-xmax, xmax+1, 2))
y_ticks = np.append(np.arange(0, 1.1, 0.2), sigmoid(0))
ax.set_yticks(np.unique(np.sort(y_ticks)))
ax.set_xlim(-xmax, xmax)
ax.set_ylim(-.1, 1.1)
ax.set_xlabel('x')
ax.set_ylabel('PROBABILITY')
ax.set_title('Sigmoid Function')
plt.legend(fontsize=20, loc='upper left')
plt.savefig("030.png")
plt.close()

In [34]:
fig, ax = plt.subplots()

xplot = np.linspace(-xmax, xmax, 200)
ax.plot(xplot, sigmoid(xplot), c='#CBC3E3', linewidth=4, label='$\\frac{1}{1 + e^{-x}}$')
ax.axvline(1, ymin=-.1, ymax=(sigmoid(1)+.1) / 1.2, color='grey', linestyle='--', linewidth=1)
ax.axhline(sigmoid(1), xmin=0, xmax=7/12, color='grey', linestyle='--', linewidth=1)
ax.axvline(2, ymin=0, ymax=(sigmoid(2)+.1) / 1.2, color='grey', linestyle='--', linewidth=1)
ax.axhline(sigmoid(2), xmin=0, xmax=8/12, color='grey', linestyle='--', linewidth=1)

ax.set_xticks(np.arange(-xmax, xmax+1, 2))
y_ticks = np.arange(0, 1.1, 0.2)
ax.set_yticks(np.unique(np.sort(y_ticks)))
ax.set_xlim(-xmax, xmax)
ax.set_ylim(-.1, 1.1)
ax.set_xlabel('x')
ax.set_ylabel('PROBABILITY')
ax.set_title('Sigmoid Function')
plt.savefig("031.png")
plt.close()

In [35]:
fig, ax = plt.subplots()
xplot = np.linspace(-xmax, xmax, 200)
ax.plot(xplot, sigmoid(xplot), c='#CBC3E3', linewidth=4, label='$\\frac{1}{1 + e^{-x}}$')
ax.axvline(4, ymin=-.1, ymax=(sigmoid(4)+.1) / 1.2, color='grey', linestyle='--', linewidth=1)
ax.axhline(sigmoid(4), xmin=0, xmax=10/12, color='grey', linestyle='--', linewidth=1)
ax.axvline(5, ymin=0, ymax=(sigmoid(5)+.1) / 1.2, color='grey', linestyle='--', linewidth=1)
ax.axhline(sigmoid(5), xmin=0, xmax=11/12, color='grey', linestyle='--', linewidth=1)

ax.set_xticks(np.arange(-xmax, xmax+1, 2))
y_ticks = np.arange(0, 1.1, 0.2)
ax.set_yticks(np.unique(np.sort(y_ticks)))
ax.set_xlim(-xmax, xmax)
ax.set_ylim(-.1, 1.1)
ax.set_xlabel('x')
ax.set_ylabel('PROBABILITY')
ax.set_title('Sigmoid Function')
plt.savefig("032.png")
plt.close()

## CDF of Normal

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

cdf = lambda x : norm.cdf(x)

fig, ax = plt.subplots()
xplot = np.linspace(-10, 10, 200)
ax.plot(xplot, cdf(xplot), c='#CBC3E3', linewidth=4, label='$\\frac{1}{\\sqrt{2\\pi}} \\int_{-\\infty}^x e^{\\frac{-(u-\\mu)^2}{2\\sigma^2}} du$')
ax.axvline(0, color='grey', linestyle='--', linewidth=0.5)
ax.axhline(cdf(0), color='grey', linestyle='--', linewidth=0.5)
y_ticks = np.append(np.arange(0, 1.1, 0.2), cdf(0))
ax.set_yticks(np.unique(np.sort(y_ticks)))
ax.set_xlim(-6, 6)
ax.set_ylim(-.1, 1.1)
ax.set_xlabel('x')
ax.set_ylabel('PROBABILITY')
ax.set_title('CDF of Standard Normal Distribution')
ax.legend(loc='upper left', fontsize=17)
fig.savefig("033.png")
plt.close()

## Probabilities

In [37]:
X, Y = generate_line_data(DATA, 1, -1, 0)

xplot = np.linspace(-3, 3, 100)
plt.scatter(X[:,0],X[:,1],color=COLORS[Y].tolist(), s=400, alpha=.5)
plt.plot(xplot, xplot, c='b', linestyle='dashed', label="$x_1 - x_2 = 0$")
plt.scatter(0, 0, color='g', s=400, alpha=1, label='P(purple) = $\\frac{1}{2}$')
plt.xlim(xlim_min, xlim_max)
plt.ylim(ylim_min, ylim_max)
plt.xlabel("$x_1$", fontsize=15)
plt.ylabel("$x_2$", fontsize=15)
plt.legend(fontsize=15)
plt.savefig("034.png")
plt.close()

In [38]:
X, Y = generate_line_data(DATA, 1, -1, 0)


xplot = np.linspace(-3, 3, 100)
plt.scatter(X[:,0],X[:,1],color=COLORS[Y].tolist(), s=400, alpha=.5)
plt.plot(xplot, xplot, c='b', linestyle='dashed', label="$x_1 - x_2 = 0$")
plt.scatter(2, 1, color='#BF5700', s=400, alpha=1, edgecolor='black', label='(2,1) → $x_1 - x_2 = 1$')
plt.xlim(xlim_min, xlim_max)
plt.ylim(ylim_min, ylim_max)
plt.xlabel("$x_1$", fontsize=15)
plt.ylabel("$x_2$", fontsize=15)
plt.legend(fontsize=15, loc='upper left')
plt.savefig("035.png")
plt.close()

In [39]:
X, Y = generate_line_data(DATA, 1, -1, 0)

xplot = np.linspace(-3, 3, 100)
plt.scatter(X[:,0],X[:,1],color=COLORS[Y].tolist(), s=400, alpha=.5)
plt.plot(xplot, xplot, c='b', linestyle='dashed', label="$x_1 - x_2 = 0$")
plt.scatter(2, 1, color='#BF5700', s=400, alpha=1, edgecolor='black', label='(2,1) → $\\frac{1}{1 + e^{x_1 - x_2}} = .73$')
plt.xlim(xlim_min, xlim_max)
plt.ylim(ylim_min, ylim_max)
plt.xlabel("$x_1$", fontsize=15)
plt.ylabel("$x_2$", fontsize=15)
plt.legend(fontsize=15, loc='upper left')
plt.savefig("036.png")
plt.close()

In [40]:
X, Y = generate_line_data(DATA, 1, -1, 0)

xplot = np.linspace(-3, 3, 100)
plt.scatter(X[:,0],X[:,1],color=COLORS[Y].tolist(), s=400, alpha=.5)
plt.plot(xplot, xplot, c='b', linestyle='dashed', label="$x_1 - x_2 = 0$")
plt.scatter(2, 1, color='#BF5700', s=400, alpha=1, edgecolor='black', label='(2,1) → P(purple) = .73')
plt.xlim(xlim_min, xlim_max)
plt.ylim(ylim_min, ylim_max)
plt.xlabel("$x_1$", fontsize=15)
plt.ylabel("$x_2$", fontsize=15)
plt.legend(fontsize=15, loc='upper left')
plt.savefig("037.png")
plt.close()

In [41]:
X, Y = generate_line_data(DATA, 1, -1, 0)

xplot = np.linspace(-3, 3, 100)
plt.scatter(X[:,0],X[:,1],color=COLORS[Y].tolist(), s=400, alpha=.5)
plt.plot(xplot, xplot, c='b', linestyle='dashed', label="$x_1 - x_2 = 0$")
plt.scatter(1, -1, color='g', s=400, alpha=1, label='P(purple) > $\\frac{1}{2}$')
plt.xlim(xlim_min, xlim_max)
plt.ylim(ylim_min, ylim_max)
plt.xlabel("$x_1$", fontsize=15)
plt.ylabel("$x_2$", fontsize=15)
plt.legend(fontsize=15)
plt.savefig("038.png")
plt.close()

In [42]:
X, Y = generate_line_data(DATA, 1, -1, 0)

xplot = np.linspace(-3, 3, 100)
plt.scatter(X[:,0],X[:,1],color=COLORS[Y].tolist(), s=400, alpha=.5)
plt.plot(xplot, xplot, c='b', linestyle='dashed', label="$x_1 - x_2 = 0$")
plt.scatter(-1, 1, color='g', s=400, alpha=1, label='P(purple) < $\\frac{1}{2}$')
plt.xlim(xlim_min, xlim_max)
plt.ylim(ylim_min, ylim_max)
plt.xlabel("$x_1$", fontsize=15)
plt.ylabel("$x_2$", fontsize=15)
plt.legend(fontsize=15)
plt.savefig("039.png")
plt.close()

In [43]:
C = 1
def f(x1, x2):
    return C * x1 - C * x2

X, Y = generate_line_data(DATA, 1, -1, 0)

xplot = np.linspace(-6, 6, 100)
x, y = np.meshgrid(xplot, xplot)
plt.scatter(X[:,0], X[:,1], color=COLORS[Y].tolist(), s=400, alpha=.5)
plt.plot(xplot, xplot, c='b', linestyle='dashed', label="$x_1 - x_2 = 0$")
plt.contour(x, y, sigmoid(f(x, y)) - .7, [0])
plt.xlim(xlim_min, xlim_max)
plt.ylim(ylim_min, ylim_max)
plt.xlabel("$x_1$", fontsize=15)
plt.ylabel("$x_2$", fontsize=15)
plt.legend(fontsize=15)
plt.savefig("040.png")
plt.close()

In [44]:
def f(x1, x2, C):
    return C * x1 - C * x2

X, Y = generate_line_data(DATA, 1, -1, 0)

FRAMES = 30

def snap(C):
    xplot = np.linspace(-6, 6, 100)
    x, y = np.meshgrid(xplot, xplot)
    fig, ax = plt.subplots()
    ax.scatter(X[:,0], X[:,1], color=COLORS[Y].tolist(), s=400, alpha=.5)
    ax.plot(xplot, xplot, c='b', linestyle='dashed', label=r'$C x_1 - C x_2 = 0$'.replace("C", f'{C:.2f}'))
    ax.contourf(x, y, sigmoid(f(x, y, C)), alpha=.4, cmap=CMAP, vmin=0, vmax=1)
    fig.colorbar(mpl.cm.ScalarMappable(cmap=CMAP), ax=ax)
    ax.set_xlim(xlim_min, xlim_max)
    ax.set_ylim(ylim_min, ylim_max)
    ax.set_xlabel("$x_1$", fontsize=15)
    ax.set_ylabel("$x_2$", fontsize=15)
    fig.legend(fontsize=15)
    fig.savefig(TEMPFILE_NAME, bbox_inches='tight')
    plt.close()

    return im.fromarray(np.asarray(im.open(TEMPFILE_NAME)))

images = []

for i in range(0, 80):
    images.append(snap((100 - i)/100))

save_to_gif("041.gif", images, 50)

In [45]:
X, Y = generate_line_data(DATA, 1, -1, 0)

images = []

for i in range(10, 50):
    images.append(snap(i/10))

save_to_gif("042.gif", images, 100)

In [46]:
def f(x1, x2, C):
    return C * x1 - C * x2

X, Y = generate_line_data(DATA, 1, -1, 0)

def snap(C, rot):
    xplot = np.linspace(-3, 3, 100)
    x, y = np.meshgrid(xplot, xplot)
    fig = plt.figure(figsize =(14, 9), constrained_layout=True, facecolor=None)
    ax = plt.axes(projection ='3d')
    ax.view_init(20, -135 - rot)
    ax.scatter(X[:,0], X[:,1], sigmoid(f(X[:,0], X[:,1], C)), color=COLORS[Y].tolist(), s=400, alpha=.5)
    ax.plot_surface(x, y, sigmoid(f(x, y, C)), cmap=CMAP, alpha=.4, lw=0)
    ax.set_xlabel('$x_1$', fontsize=15)
    ax.set_ylabel('$x_2$', fontsize=15)
    ax.set_title(r'$\sigma(C x_1 - C x_2)$'.replace("C", f'{C:.2f}'), fontsize=40)
    ax.set_xlim(xlim_min, xlim_max)
    ax.set_ylim(ylim_min, ylim_max)
    ax.set_zlim(-.01, 1.01)
    fig.savefig(TEMPFILE_NAME)
    plt.close()

    return im.fromarray(np.asarray(im.open(TEMPFILE_NAME)))

images = []

for i in range(1, 360):
    images.append(snap(i/100, 0))

for i in range(90):
    images.append(snap(3.6, 2 * i))

save_to_gif("043.gif", images, 10)

In [47]:
def f(x1, x2, C):
    return C * x1 - C * x2

X, Y = generate_line_data(DATA, 1, -1, 0)

def snap(C, rot_z, rot_xy, rot_ang):
    xplot = np.linspace(-3, 3, 100)
    x, y = np.meshgrid(xplot, xplot)
    fig = plt.figure(figsize =(14, 9), constrained_layout=True, facecolor=None)
    ax = plt.axes(projection ='3d')
    ax.view_init(20 + rot_z, -135 - rot_ang, rot_xy)
    ax.scatter(X[:,0], X[:,1], sigmoid(f(X[:,0], X[:,1], C)), color=COLORS[Y].tolist(), s=400, alpha=.5)
    ax.plot_surface(x, y, sigmoid(f(x, y, C)), cmap=CMAP, alpha=.4, lw=0)
    ax.plot(xplot, xplot, np.zeros_like(xplot) + .5, linestyle='--', linewidth=3, color='b', label='$3.6x_1 - 3.6x_2 = 0$')
    ax.set_xlabel('$x_1$', fontsize=15)
    ax.set_ylabel('$x_2$', fontsize=15)
    ax.set_title(r'$\sigma(C x_1 - C x_2)$'.replace("C", f'{C:.2f}'), fontsize=40)
    ax.set_xlim(xlim_min, xlim_max)
    ax.set_ylim(ylim_min, ylim_max)
    ax.set_zlim(-.01, 1.01)
    ax.legend(fontsize=15, loc='upper left')
    fig.savefig(TEMPFILE_NAME)
    plt.close()

    return im.fromarray(np.asarray(im.open(TEMPFILE_NAME)))

images = []

for i in range(45):
    images.append(snap(3.6, 0, 0, i))

for i in range(45):
    images.append(snap(3.6, 0, 0, 45 - i))

for i in range(70):
    images.append(snap(3.6, i, 0, 0))

for i in range(1, 45):
    images.append(snap(3.6, 70, - i, 0))

for _ in range(15):
    images.append(snap(3.6, 70, -45, 0))

save_to_gif("044.gif", images, 10)

## Probability of 8/10 Heads

In [48]:
def f(p):
    return p**8 * (1 - p)**2

prob_p = .5
ymin = -.0001
ymax = f(.8)+.0001

p = np.linspace(0, 1, 400)
plt.plot(p, f(p), linewidth=3)
plt.axvline(prob_p, ymin=ymin, ymax=f(prob_p)/ymax, color='grey', linestyle='--', linewidth=1, label='p = ' + f'{prob_p:.1f}')
plt.axhline(f(prob_p), xmin=0, xmax=prob_p,  color='grey', linestyle='--', linewidth=1, label='P(8 / 10 Heads) = ' + f'{f(prob_p):.3f}')
plt.xlabel('p')
plt.ylabel('P(8 / 10 Heads)')
plt.title('P(8/10 Heads) as a function of p')
plt.xlim(0,1)
plt.ylim(ymin, ymax)
plt.legend()
plt.savefig("045.png")
plt.close()

In [49]:
def f(p):
    return p**8 * (1 - p)**2

prob_p = .6

p = np.linspace(0, 1, 400)
plt.plot(p, f(p), linewidth=3)
plt.axvline(prob_p, ymin=ymin, ymax=f(prob_p)/(ymax + ymin), color='grey', linestyle='--', linewidth=1, label='p = ' + f'{prob_p:.1f}')
plt.axhline(f(prob_p), xmin=0, xmax=prob_p,  color='grey', linestyle='--', linewidth=1, label='P(8 / 10 Heads) = ' + f'{f(prob_p):.3f}')
plt.xlabel('p')
plt.ylabel('P(8 / 10 Heads)')
plt.title('P(8/10 Heads) as a function of p')
plt.xlim(0,1)
plt.ylim(ymin, ymax)
plt.legend()
plt.savefig("046.png")
plt.close()

In [50]:
def f(p):
    return p**8 * (1 - p)**2

prob_p = .7

p = np.linspace(0, 1, 400)
plt.plot(p, f(p), linewidth=3)
plt.axvline(prob_p, ymin=ymin, ymax=f(prob_p)/(ymax + ymin), color='grey', linestyle='--', linewidth=1, label='p = ' + f'{prob_p:.1f}')
plt.axhline(f(prob_p), xmin=0, xmax=prob_p,  color='grey', linestyle='--', linewidth=1, label='P(8 / 10 Heads) = ' + f'{f(prob_p):.3f}')
plt.xlabel('p')
plt.ylabel('P(8 / 10 Heads)')
plt.title('P(8/10 Heads) as a function of p')
plt.xlim(0,1)
plt.ylim(ymin, ymax)
plt.legend()
plt.savefig("047.png")
plt.close()

## Maximization

In [51]:
def df(p):
    return 8*p**7 * (1-p)**2 - 2*p**8 * (1-p)

p_max = .8
y_max = f(p_max)
slope = df(p_max)

def tangent(p):
    return slope * (p - p_max) + y_max

p = np.linspace(0, 1, 400)
tan_p = np.linspace(.7, .9, 50)

plt.plot(p, f(p), label='P(8 out of 10 Heads) = $p^8(1-p)^2$')
plt.plot(tan_p, tangent(tan_p), label='Tangent at max', linestyle='--')
plt.scatter(p_max, y_max, color='red', label=f'Max at (p={p_max:.2f})')
plt.axvline(0.8, color='grey', linestyle='--', linewidth=0.5)
plt.xlabel('p')
plt.ylabel('P(8 out of 10 Heads)')
plt.legend(loc='upper left')
plt.savefig("048.png")
plt.close()

## Training 2D

In [52]:
X, Y = generate_line_data(DATA, 1, -1, 0)
class LR():

    def __init__(self, model):
        self.model = model

    def _snap_learning(self, X, Y, filename):
        h = .02
        x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
        y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
        xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                             np.arange(y_min, y_max, h))
        meshData = np.c_[xx.ravel(), yy.ravel()]

        fig, ax = plt.subplots(frameon=False)
        ax.scatter(X[:, 0], X[:, 1], color=COLORS[Y].tolist(), s=100, alpha=.9)
        Z = self.model.predict(meshData)
        Z = np.array([z[0] for z in Z]).reshape(xx.shape)
        ax.contourf(xx, yy, Z, alpha=.4, cmap=CMAP)
        fig.savefig(filename + '.png')
        plt.close()
        return np.asarray(im.open(filename + '.png'))

    def animate_learning(self, X, Y_0_1, snap_freq=10, filename='learn', duration=1000, **kwargs):
        images = []
        if 'epochs' in kwargs:
            epochs = kwargs['epochs']
            kwargs.pop('epochs', None)
        else:
            epochs = snap_freq

        for _ in range(int(epochs / snap_freq)):
            self.model.fit(X, Y_0_1, epochs=snap_freq, **kwargs)
            images.append(im.fromarray(self._snap_learning(X, Y, filename)))

        images[0].save(
            filename + '.gif',
            optimize=False,
            save_all=True,
            append_images=images[1:],
            loop=0,
            duration=duration
        )
        return self.model

Y_0_1 = Y.copy()
Y_0_1[np.where(Y_0_1 == 0)] = 1
Y_0_1[np.where(Y_0_1 == 2)] = 0

model = keras.models.Sequential()
model.add(keras.layers.Dense(1, input_dim=2, activation="sigmoid"))
model.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=1e-1))

obj = LR(model)
obj.animate_learning(X, Y_0_1, 1, '049', 200, epochs=40, batch_size=100)

/Users/lance/Documents/BostonUniversity/Data-Science-Fundamentals/env/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.6011  
3335/3335 ━━━━━━━━━━━━━━━━━━━━ 1s 371us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.3574 
3335/3335 ━━━━━━━━━━━━━━━━━━━━ 1s 402us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 1.1249 
3335/3335 ━━━━━━━━━━━━━━━━━━━━ 1s 369us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.9345 
3335/3335 ━━━━━━━━━━━━━━━━━━━━ 1s 397us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.7674 
3335/3335 ━━━━━━━━━━━━━━━━━━━━ 1s 347us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.6241
3335/3335 ━━━━━━━━━━━━━━━━━━━━ 1s 382us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.5191 
3335/3335 ━━━━━━━━━━━━━━━━━━━━ 1s 391us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.4282 
3335/3335 ━━━━━━━━━━━━━━━━━━━━ 1s 368us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.3585 
3335/3335 ━━━━━━━━━━━━━━━━━━━━ 1s 369us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.3066 
3335/3335 ━━━━━━━━━━━━━━━━━━━━ 1s 365us/step
2/2 ━━━━━━━━━━━━━━━━

<Sequential name=sequential, built=True>

## Training 3D

In [53]:
X, Y = generate_line_data(DATA, 1, -1, 0)

class LR():

    def __init__(self, model):
        self.model = model

    def _snap_learning_3D(self, X, Y_0_1, filename, rot):
        xplot = np.linspace(-3, 3, 100)
        x, y = np.meshgrid(xplot, xplot)
        meshData = np.c_[x.ravel(), y.ravel()]
        fig = plt.figure(figsize =(14, 9), constrained_layout=True, facecolor=None)
        ax = plt.axes(projection ='3d')
        ax.view_init(15, - 25 - rot)
        ax.scatter(X[:,0], X[:,1], Y_0_1, color=COLORS[Y].tolist(), s=400, alpha=.5)
        ax.plot_surface(x, y, self.model.predict(meshData).reshape(x.shape), alpha=.2, cmap=CMAP, lw=0)
        ax.set_xlim(xlim_min, xlim_max)
        ax.set_ylim(ylim_min, ylim_max)
        fig.savefig(filename + '.png')
        plt.close()
        return np.asarray(im.open(filename + '.png'))


    def animate_learning_3D(self, X, Y_0_1, snap_freq=10, filename='learn', duration=1000, **kwargs):
        images = []
        if 'epochs' in kwargs:
            epochs = kwargs['epochs']
            kwargs.pop('epochs', None)
        else:
            epochs = snap_freq

        for i in range(int(epochs / snap_freq)):
            self.model.fit(X, Y_0_1, epochs=snap_freq, **kwargs)
            images.append(im.fromarray(self._snap_learning_3D(X, Y_0_1, filename, 0)))
        
        for i in range(20):
            images.append(im.fromarray(self._snap_learning_3D(X, Y_0_1, filename, i)))

        images[0].save(
            filename + '.gif',
            optimize=False,  # important for transparent background
            save_all=True,
            append_images=images[1:],
            loop=0,
            duration=duration
        )
        return self.model

Y_0_1 = Y.copy()
Y_0_1[np.where(Y_0_1 == 0)] = 1
Y_0_1[np.where(Y_0_1 == 2)] = 0

model = keras.models.Sequential()
model.add(keras.layers.Dense(1, input_dim=2, activation="sigmoid"))
model.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=1e-1))

obj = LR(model)
obj.animate_learning_3D(X, Y_0_1, 1, '050', 200, epochs=50, batch_size=100)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3028  
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 883us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2516 
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 583us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2069 
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 559us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1820 
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 716us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1505 
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 604us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1311 
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 658us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1123 
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 614us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0994 
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 577us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0908 
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 795us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0805 
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 736us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - 

<Sequential name=sequential_1, built=True>

## Sigmoid with boundary

In [ ]:
def f(x1, x2, C):
    return C * x1 - C * x2

X, Y = generate_line_data(DATA, 1, -1, 0)

def snap(C, rot_z, rot_xy, rot_ang):
    xplot = np.linspace(-3, 3, 100)
    x, y = np.meshgrid(xplot, xplot)
    fig = plt.figure(figsize =(14, 9), constrained_layout=True, facecolor=None)
    ax = plt.axes(projection ='3d')
    ax.view_init(20 + rot_z, -135 - rot_ang, rot_xy)
    ax.scatter(X[:,0], X[:,1], sigmoid(f(X[:,0], X[:,1], C)), color=COLORS[Y].tolist(), s=400, alpha=.5)
    ax.plot_surface(x, y, sigmoid(f(x, y, C)), cmap=CMAP, alpha=.4, lw=0)
    ax.plot(xplot, xplot, np.zeros_like(xplot) + .5, linestyle='--', linewidth=3, color='b', label='$3.6x_1 - 3.6x_2 = 0$')
    ax.set_xlabel('$x_1$', fontsize=15)
    ax.set_ylabel('$x_2$', fontsize=15)
    ax.set_title(r'$\sigma(C x_1 - C x_2)$'.replace("C", f'{C:.2f}'), fontsize=40)
    ax.set_xlim(xlim_min, xlim_max)
    ax.set_ylim(ylim_min, ylim_max)
    ax.set_zlim(-.01, 1.01)
    ax.legend(fontsize=15, loc='upper left')
    fig.savefig(TEMPFILE_NAME)
    plt.close()

    return im.fromarray(np.asarray(im.open(TEMPFILE_NAME)))

images = []

for i in range(45):
    images.append(snap(3.6, 0, 0, i))

for i in range(45):
    images.append(snap(3.6, 0, 0, 45 - i))

for i in range(70):
    images.append(snap(3.6, i, 0, 0))

for i in range(1, 45):
    images.append(snap(3.6, 70, - i, 0))

for _ in range(15):
    images.append(snap(3.6, 70, -45, 0))

save_to_gif("051.gif", images, 10)